In [ ]:
CREATE OR REPLACE API INTEGRATION github
    api_provider = git_https_api
    api_allowed_prefixes = ('https://github.com/<your_profile>')
    enabled = true
    allowed_authentication_secrets = all

In [ ]:
CREATE OR REPLACE SECURITY INTEGRATION DBT_CLOUD
  TYPE = OAUTH
  ENABLED = TRUE
  OAUTH_CLIENT = CUSTOM
  OAUTH_CLIENT_TYPE = 'CONFIDENTIAL'
  OAUTH_REDIRECT_URI = '<REDIRECT_URI>'
  OAUTH_ISSUE_REFRESH_TOKENS = TRUE
  OAUTH_REFRESH_TOKEN_VALIDITY = 7776000;

In [ ]:
with

integration_secrets as (
  select parse_json(system$show_oauth_client_secrets('DBT_CLOUD')) as secrets
)

select
  secrets:"OAUTH_CLIENT_ID"::string     as client_id,
  secrets:"OAUTH_CLIENT_SECRET"::string as client_secret
from
  integration_secrets;

In [ ]:
USE SCHEMA FLIGHTS.PUBLIC;

CREATE OR REPLACE STAGE s3_stage
  URL='s3://project-flight-data-bucket/'
  CREDENTIALS=(AWS_KEY_ID='<key>' AWS_SECRET_KEY='<secret>');

In [ ]:
CREATE OR REPLACE FILE FORMAT csv_format
  TYPE = 'CSV'
  FIELD_DELIMITER = '|'
  SKIP_HEADER = 1
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  ESCAPE_UNENCLOSED_FIELD = None
  NULL_IF = ('NULL', 'null', '');


In [ ]:
CREATE OR REPLACE TABLE s3 (
    airport_icao STRING,
    timestamp TIMESTAMP,
    fromdate TIMESTAMP,
    todate TIMESTAMP,
    response_code INTEGER,
    response VARIANT
);

In [ ]:
COPY INTO s3
FROM @s3_stage
FILE_FORMAT = csv_format
ON_ERROR = 'CONTINUE';